In [1]:
from pymongo import MongoClient
import pprint
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import copy
import re
from time import sleep
from selenium import webdriver
import numpy as np

In [2]:
carmax_url = 'https://www.carmax.com/cars/ford'
#carmax_url = 'https://www.carmax.com/'
#carmax_url = 'https://www.carvana.com/cars/ford'
#headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

In [ ]:
client = MongoClient('localhost', 27017)
db = client['capstone1']
pages = db['Ford_TX']

In [3]:
#Ford_files=['Ford_2013_2016.html','Ford_2017_2020.html']
Ford_files = ['Ford_NY.html']

In [4]:
url_links = []
for file in Ford_files:
    soup = BeautifulSoup(open("/home/ali/Documents/CapStone_Projects/CapStone1/" + file), "html.parser")
    for idx,link in enumerate(soup.find_all('article','car-tile')):
        url_links.append(link.a['href'])
        

In [9]:
car_mileage = []
car_price = []
car_mpg = []
car_model_year = []
car_model = []
car_make = []
car_color = []
car_transmission = []
car_engine_size = []
car_horse_power = []
car_engine_cylinder = []
car_msrp_price = []

url_links = list(set(url_links))

In [10]:
i=0

for link in url_links[301:-1]:
    scrape_url = link
    
    print(link,i)
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}
    
    r_1 = requests.get(scrape_url,headers=headers)
    
    soup = BeautifulSoup(r_1.text,'html.parser')
    
    try:
        car_mileage.append(soup.find_all('div','value')[1].text.strip())
    except Exception as e:
        car_mileage.append(np.nan)
        
    try:    
        car_price.append(soup.find_all('div','value')[0].text.strip())
    except Exception as e:
        car_price.append(np.nan)
    
    try:  
        car_mpg.append([np.nan if soup.find_all('div','mpg')==[] else soup.find('div','mpg').text])
    except Exception as e:
        car_mpg.append(np.nan)
        
    try:
        car_model_year.append(soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__year-make').text.strip(" ").split(" ",1)[0])
    except Exception as e:
        car_model_year.append(np.nan)
        
    try:
        car_model.append(soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__year-make').text.strip(" ").split(" ",1)[1])
    except Exception as e:
        car_model.append(np.nan)
    
    try:
        car_make.append(soup.find('div','price-mileage__car-title').find('span','price-mileage__car-title__model-trim').text.strip())
    except Exception as e:
        car_make.append(np.nan)
    
    a = soup.find_all('script')[6]
    a = str(a)
    
    try:
        car_color.append(a.split('ExteriorColor":"')[1].split('"')[0])
    except Exception as e:
        car_color.append(np.nan)
    
    try:
        car_transmission.append(a.split('Transmission":"')[1].split('"')[0])
    except Exception as e:
        car_transmission.append(np.nan)
    
    try:
        car_engine_size.append(float(a.split('EngineSize":"')[1].split('L')[0]))
    except Exception as e:
        car_engine_size.append(np.nan)
    
    
    
    if a.split('Horsepower":')[1].split(',')[0] != 'null':
        var_HorsePower = float(a.split('Horsepower":')[1].split(',')[0])
    else:
        var_HorsePower = np.nan  
    car_horse_power.append(var_HorsePower)
    
    
    if a.split('EngineCylinders":')[1].split(',')[0] != 'null':
        car_EngineCylinder = int(a.split('EngineCylinders":')[1].split(',')[0])
    else:
        car_EngineCylinder = np.nan
    car_engine_cylinder.append(car_EngineCylinder)
    
    try:
        car_msrp_price.append(float(a.split('Msrp":')[1].split(',')[0]))
    except Exception as e:
        car_msrp_price.append(np.nan)
    
#    pages.insert_one({'car_mileage': car_mileage, 
#                      'car_price': car_price, 
#                      'car_mpg': car_mpg ,
#                      'car_model_year': car_model_year,
#                      'car_model': car_model ,
#                      'car_make': car_make,
#                      'car_color': car_color,
#                      'car_transmission': car_transmission,
#                      'car_engine_size': car_engine_size,
#                      'car_horse_power': car_horse_power ,
#                      'car_engine_cylinder': car_engine_cylinder,
#                      'car_msrp_price':car_msrp_price})
    
    
    i += 1
    sleep(8)
    #if i>200:
    #    break 

https://www.carmax.com/car/19975043 0
https://www.carmax.com/car/19974547 1
https://www.carmax.com/car/19912408 2
https://www.carmax.com/car/19410675 3
https://www.carmax.com/car/19402181 4
https://www.carmax.com/car/20063050 5
https://www.carmax.com/car/20205627 6
https://www.carmax.com/car/18236958 7
https://www.carmax.com/car/19996894 8
https://www.carmax.com/car/19706622 9
https://www.carmax.com/car/20013499 10
https://www.carmax.com/car/20102409 11
https://www.carmax.com/car/20063494 12
https://www.carmax.com/car/20068003 13
https://www.carmax.com/car/19441871 14
https://www.carmax.com/car/19891955 15
https://www.carmax.com/car/19479109 16
https://www.carmax.com/car/20102410 17
https://www.carmax.com/car/19124302 18
https://www.carmax.com/car/20111046 19
https://www.carmax.com/car/18843809 20
https://www.carmax.com/car/20013801 21
https://www.carmax.com/car/20025703 22
https://www.carmax.com/car/19542805 23
https://www.carmax.com/car/19634801 24
https://www.carmax.com/car/19788525

https://www.carmax.com/car/19543045 208
https://www.carmax.com/car/19667901 209
https://www.carmax.com/car/17914598 210
https://www.carmax.com/car/20062645 211
https://www.carmax.com/car/19521172 212
https://www.carmax.com/car/20180457 213
https://www.carmax.com/car/19989517 214
https://www.carmax.com/car/19891960 215
https://www.carmax.com/car/19989883 216
https://www.carmax.com/car/19619179 217
https://www.carmax.com/car/20013001 218
https://www.carmax.com/car/20153752 219
https://www.carmax.com/car/19927692 220
https://www.carmax.com/car/20099429 221
https://www.carmax.com/car/19555448 222
https://www.carmax.com/car/20102114 223
https://www.carmax.com/car/20025920 224
https://www.carmax.com/car/20127453 225
https://www.carmax.com/car/20160127 226
https://www.carmax.com/car/19501194 227
https://www.carmax.com/car/19526488 228
https://www.carmax.com/car/19310865 229
https://www.carmax.com/car/20047022 230
https://www.carmax.com/car/20115149 231
https://www.carmax.com/car/19913790 232


In [11]:
df = pd.DataFrame({'car_mileage': car_mileage, 
                   'car_price': car_price, 
                   'car_mpg': car_mpg ,
                   'car_model_year': car_model_year,
                   'car_model': car_model ,
                   'car_make': car_make,
                   'car_color': car_color,
                   'car_transmission': car_transmission,
                   'car_engine_size': car_engine_size,
                   'car_horse_power': car_horse_power ,
                   'car_engine_cylinder': car_engine_cylinder,
                   'car_msrp_price':car_msrp_price,
                                          })
df.tail()

,car_mileage,car_price,car_mpg,car_model_year,car_model,car_make,car_color,car_transmission,car_engine_size,car_horse_power,car_engine_cylinder,car_msrp_price
243,94K miles,"$8,998*",[26/36 mpg],2014,Focus SE,Ford,Red,Manual 5 Speed,2.0,160.0,4.0,18700.0
244,75K miles,"$18,998*",[18/27 mpg],2017,Mustang,Ford,Gray,Automatic,3.7,300.0,6.0,25200.0
245,33K miles,"$40,998*",[nan],2020,F150 Lariat,Ford,Brown,Automatic,5.0,NaN,NaN,38800.0
246,81K miles,"$13,998*",[22/30 mpg],2013,Escape Titanium,Ford,Black,Automatic,2.0,231.0,4.0,30400.0
247,24K miles,"$32,998*",[nan],2020,F150 XLT,Ford,Gray,Automatic,5.0,NaN,8.0,35300.0


In [12]:
df.to_csv("Ford_NY_2.csv")

In [ ]:
#xxx= pd.read_csv('Ford_TX.csv')

In [ ]:
#xxx